In [13]:
import os
from openai import AzureOpenAI
import PyPDF2

def get_json(file_path):

    client = AzureOpenAI(
      azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
      api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
      api_version="2024-02-01"
    )

    # file_path = "company/VLO-3Q23-Earnings-Release.pdf"

    # def extract_text_from_pdf(pdf_path):
    #     text = ""
    #     with open(pdf_path, "rb") as file:
    #         pdf_reader = PyPDF2.PdfReader(file)
    #         for page in pdf_reader.pages:
    #             page_text = page.extract_text()
    #             if page_text:
    #                 text += page_text
    #     return text

    # input_context = extract_text_from_pdf(file_path)
    
    input_context = ""
    pdf_reader = PyPDF2.PdfReader(file_path)
    for page in pdf_reader.pages:
        page_text = page.extract_text()
        if page_text:
            input_context += page_text

    
    instructions = """
        As a Financial Analyst, you will leverage your expertise to generate tailored Financial Analysis Reports that cater to the specific requirements of clients. This role involves in-depth analysis of financial statements and market data to uncover insights into a company's financial performance and stability.
    You will engage directly with clients to gather essential information and refine the report based on their feedback, ensuring that the final product precisely meets their needs and expectations.

    Key Objectives:

    Analytical Precision: Utilize analytical skills to interpret financial data, identify trends, and detect anomalies.
    Effective Communication: Simplify and convey complex financial information in a clear and actionable manner for non-specialist audiences.
    Client Focus: Tailor reports dynamically based on client feedback, aligning the analysis with their strategic goals.
    Quality Assurance: Maintain the highest standards of quality and integrity in report generation, adhering to established benchmarks for analytical rigor.
    Performance Indicators:

    The effectiveness of the Financial Analysis Report is measured by its ability to provide actionable insights that support corporate decision-making, identify areas for operational improvement, and evaluate the company's financial health. Success is reflected in the report's contribution to informed investment decisions and strategic planning.
    """

    user_content = f"""The report will include the following sections and I want to extract the capital expenditure and other metric like 

    Operating Activities "Change in Working Capital",
            "Net Cash from Operating Activities"
    Investing Activities has "Acquisition of Fixed Assets & Intangibles",
            "Net Cash from Investing Activities"
    Financing Activities includes "Dividends Paid",
            "Cash from (Repayment of) Debt",
            "Net Cash from Financing Activities",
    Net Change includes  Net Change in Cash
    Metadata includes "Report Date", "Publish Date", "Source"
    Profitability Metrics like "EBITDA",
            "Gross Profit Margin",
            "Operating Margin",
            "Net Profit Margin",
            "Return on Equity",
            "Return on Assets",
            "Return On Invested Capital",
    Liquidity Metrics include Current Ratio
    Solvency Metrics like  "Total Debt", "Liabilities to Equity Ratio", "Debt Ratio",
    Cash Flow Metrics like "Free Cash Flow", "Free Cash Flow to Net Income", "Cash Return On Invested Capital",
    Other Important Metrics like "Piotroski F-Score", "Net Debt / EBITDA", "Dividend Payout Ratio
    Capital Expenditure    

    ```Document
    {input_context}
    ```
    """

    final_content = """Please provide the  company name, year and quarter, capital expenditure value in billions of US dollars in JSON format.
    use the following json structure
    ```json
    {
        "company":"",
        "year":"",
        "quarter":"",
        "capex":""    
    }
    ```
    Note: Only Json Data is required no other text is required.
    ```json

    """

    how_content = """Please provide the steps how you found or calculated capital expenditure value.
    """

    messages = [
        {"role": "system", "content": instructions},
        {"role": "user", "content": user_content}
    ]
    response = client.chat.completions.create(
        model = os.getenv("DEPLOYMENT_NAME"),
        messages = messages,
    )
    try:
        response = response.choices[0].message.content
        messages.append({"role": "assistant", "content":response.choices[0].message.content})
        messages.append({"role": "user", "content":final_content})
    except:
        messages.append({"role": "assistant", "content":response})
        messages.append({"role": "user", "content":final_content})
        # return 0
    

    response_json = client.chat.completions.create(
        model = os.getenv("DEPLOYMENT_NAME"),
        messages = messages,
    )
    messages.append({"role": "assistant", "content":response_json.choices[0].message.content})
    messages.append({"role": "user", "content":how_content})
    how_response = client.chat.completions.create(
        model = os.getenv("DEPLOYMENT_NAME"),
        messages = messages
    )
    return response_json.choices[0].message.content, how_response.choices[0].message.content


In [14]:
import os
import io
import json
import PyPDF2
from azure.storage.blob import ContainerClient

connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
container_name = "earningcalltranscriptscontainer"  # Replace with your container name
container_client = ContainerClient.from_connection_string(
    conn_str=connection_string,
    container_name=container_name
)

blob_list = container_client.list_blobs()
# stream = io.BytesIO()
# file_streams = [container_client.download_blob(blob).readall() for blob in blob_list]
# text = io.BytesIO()
# x = container_client.download_blob("AMZN-Q1-2024-Earnings-Release.pdf").readinto(text)
# text.write(x)

final_json = []
cont = 0
for blob in blob_list:
    print(f"File: {blob.name}")
    if cont == 3:
        break
    stream = io.BytesIO()
    bytes = container_client.download_blob(blob).readinto(stream)
    op, how = get_json(stream)
    cont += 1
    op = json.loads(op[op.index("{"):len(op)-op[::-1].index("}")])
    print(op, "\n\n", how)
    final_json.append(op)

    

# ntext = ""
# pdf_reader = PyPDF2.PdfReader(text)
# for page in pdf_reader.pages:
#     page_text = page.extract_text()
#     if page_text:
#         ntext += page_text
# ntext[:50]

File: 2021q1-alphabet-earnings-release.pdf


{'company': 'Alphabet Inc.', 'year': '2021', 'quarter': 'Q1', 'capex': -5.942} 

 Based on the provided financial report, the calculation of capital expenditure can be determined by analyzing the information related to purchases of property and equipment. 

Here are the steps to calculate the capital expenditure value:

1. Analyze the section of the financial report related to Investing Activities.
2. Look for the specific metric or item that represents purchases of property and equipment. In this case, it is stated as "Purchases of property and equipment" under the Investing Activities section.
3. Note the value associated with purchases of property and equipment. In this case, it is stated as $(5,942) million.
4. Convert the value from million to billion by dividing it by 1000. In this case, $(5,942) million is equivalent to $5.942 billion.
5. Record the calculated capital expenditure value.

By following these steps, the capital expenditure value of Alphabet Inc. for the specified q